# Cattle Price Analysis: a comparison between Ireland and Spain

### 0. Introduction

Agriculture 

- **Import essential libraries**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests 
import json

import warnings
warnings.filterwarnings("ignore")

### 1. Data collection

Four datasets were collected online, i.e.:
- beef price of Ireland and Spain from 2017 to 2021
- beef production of Ireland and Spain from 2017 to 2021
- beef feeding price of two stuff in Ireland from 2017 to 2021 (unfortunately the beef feeding price of Spain couldn't collect properly online
- pigmeat price of Ireland and Spain from 2017 to 2021 (only the price of pigclass E was collected)

#### 1.1 Beef price

In this section, the beef price dataset was processed for merging later.

In [2]:
beef_price = pd.read_excel("beef_price_2017_2021.xlsx")
beef_price.head()

,Year,Week,Begin Date,End Date,Member State,Category,Product,Price
0,2021,52,2021-12-27,2022-01-02,Ireland,Bulls,B R3,333.26
1,2021,52,2021-12-27,2022-01-02,Spain,Cows,D R3,304.93
2,2021,52,2021-12-27,2022-01-02,Ireland,Cows,D R3,377.86
3,2021,52,2021-12-27,2022-01-02,Spain,Heifers,E R3,424.45
4,2021,52,2021-12-27,2022-01-02,Ireland,Heifers,E R3,434.96


In [3]:
beef_price.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2088 entries, 0 to 2087
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Year          2088 non-null   int64         
 1   Week          2088 non-null   int64         
 2   Begin Date    2088 non-null   datetime64[ns]
 3   End Date      2088 non-null   datetime64[ns]
 4   Member State  2088 non-null   object        
 5   Category      2088 non-null   object        
 6   Product       2088 non-null   object        
 7   Price         2088 non-null   float64       
dtypes: datetime64[ns](2), float64(1), int64(2), object(3)
memory usage: 130.6+ KB


In [4]:
beef_price.shape

(2088, 8)

In [5]:
beef_price.isnull().sum()

Year            0
Week            0
Begin Date      0
End Date        0
Member State    0
Category        0
Product         0
Price           0
dtype: int64

In [6]:
# create a new column that shows date infomation (year-month)
beef_price["Month"] = beef_price["Begin Date"].dt.to_period("M")
beef_price.head()

,Year,Week,Begin Date,End Date,Member State,Category,Product,Price,Month
0,2021,52,2021-12-27,2022-01-02,Ireland,Bulls,B R3,333.26,2021-12
1,2021,52,2021-12-27,2022-01-02,Spain,Cows,D R3,304.93,2021-12
2,2021,52,2021-12-27,2022-01-02,Ireland,Cows,D R3,377.86,2021-12
3,2021,52,2021-12-27,2022-01-02,Spain,Heifers,E R3,424.45,2021-12
4,2021,52,2021-12-27,2022-01-02,Ireland,Heifers,E R3,434.96,2021-12


In [7]:
# create a group to generate the average monthly price of each product and each category in each country
# instead of showing the price weekly in the original dataset
# convenient for later merging also
beef_price_group = beef_price.groupby(["Member State", "Category", "Month"]).transform("mean")
beef_price_group

,Year,Week,Begin Date,End Date,Price
0,2021.0,50.5,2021-12-16 12:00:00,2021-12-22 12:00:00,321.9675
1,2021.0,50.5,2021-12-16 12:00:00,2021-12-22 12:00:00,304.8400
2,2021.0,50.5,2021-12-16 12:00:00,2021-12-22 12:00:00,378.2125
3,2021.0,50.5,2021-12-16 12:00:00,2021-12-22 12:00:00,426.5175
4,2021.0,50.5,2021-12-16 12:00:00,2021-12-22 12:00:00,431.5850
...,...,...,...,...,...
2083,2017.0,3.0,2017-01-16 00:00:00,2017-01-22 00:00:00,388.4424
2084,2017.0,3.0,2017-01-16 00:00:00,2017-01-22 00:00:00,383.6880
2085,2017.0,3.0,2017-01-16 00:00:00,2017-01-22 00:00:00,370.5560
2086,2017.0,3.0,2017-01-16 00:00:00,2017-01-22 00:00:00,361.8120


In [8]:
# add a new column of the mean price to the dataset
beef_price["price"] = beef_price_group["Price"]
beef_price

,Year,Week,Begin Date,End Date,Member State,Category,Product,Price,Month,price
0,2021,52,2021-12-27,2022-01-02,Ireland,Bulls,B R3,333.260,2021-12,321.9675
1,2021,52,2021-12-27,2022-01-02,Spain,Cows,D R3,304.930,2021-12,304.8400
2,2021,52,2021-12-27,2022-01-02,Ireland,Cows,D R3,377.860,2021-12,378.2125
3,2021,52,2021-12-27,2022-01-02,Spain,Heifers,E R3,424.450,2021-12,426.5175
4,2021,52,2021-12-27,2022-01-02,Ireland,Heifers,E R3,434.960,2021-12,431.5850
...,...,...,...,...,...,...,...,...,...,...
2083,2017,1,2017-01-02,2017-01-08,Spain,Heifers,E R3,370.450,2017-01,388.4424
2084,2017,1,2017-01-02,2017-01-08,Ireland,Heifers,E R3,382.150,2017-01,383.6880
2085,2017,1,2017-01-02,2017-01-08,Ireland,Steers,C R3,368.660,2017-01,370.5560
2086,2017,1,2017-01-02,2017-01-08,Ireland,Young bulls,A R3,361.230,2017-01,361.8120


In [9]:
# remove the prices that duplicating
beef_price = beef_price.drop_duplicates(["Member State", "Product", "price"], keep = "last")
beef_price.head(20)

,Year,Week,Begin Date,End Date,Member State,Category,Product,Price,Month,price
24,2021,49,2021-12-06,2021-12-12,Ireland,Bulls,B R3,312.81,2021-12,321.9675
25,2021,49,2021-12-06,2021-12-12,Spain,Cows,D R3,302.30,2021-12,304.8400
26,2021,49,2021-12-06,2021-12-12,Ireland,Cows,D R3,380.18,2021-12,378.2125
27,2021,49,2021-12-06,2021-12-12,Spain,Heifers,E R3,421.74,2021-12,426.5175
28,2021,49,2021-12-06,2021-12-12,Ireland,Heifers,E R3,428.07,2021-12,431.5850
29,2021,49,2021-12-06,2021-12-12,Ireland,Steers,C R3,424.52,2021-12,427.8350
30,2021,49,2021-12-06,2021-12-12,Ireland,Young bulls,A R3,406.65,2021-12,407.8750
31,2021,49,2021-12-06,2021-12-12,Spain,Young bulls,A R3,439.50,2021-12,436.4375
64,2021,44,2021-11-01,2021-11-07,Ireland,Bulls,B R3,306.72,2021-11,312.5720
65,2021,44,2021-11-01,2021-11-07,Spain,Cows,D R3,293.63,2021-11,294.8920


In [10]:
beef_price = beef_price.drop(columns = {"Year", "Week", "Begin Date", "End Date", "Price"})
beef_price.head()

,Member State,Category,Product,Month,price
24,Ireland,Bulls,B R3,2021-12,321.9675
25,Spain,Cows,D R3,2021-12,304.8400
26,Ireland,Cows,D R3,2021-12,378.2125
27,Spain,Heifers,E R3,2021-12,426.5175
28,Ireland,Heifers,E R3,2021-12,431.5850


In [11]:
beef_price.columns = ["country", "category", "product", "date", "beef price/100kg"]
new_order = [-2, 0, 1, 2, -1]
beef_price = beef_price[beef_price.columns[new_order]]
beef_price.head()

,date,country,category,product,beef price/100kg
24,2021-12,Ireland,Bulls,B R3,321.9675
25,2021-12,Spain,Cows,D R3,304.8400
26,2021-12,Ireland,Cows,D R3,378.2125
27,2021-12,Spain,Heifers,E R3,426.5175
28,2021-12,Ireland,Heifers,E R3,431.5850


In [12]:
beef_price.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 480 entries, 24 to 2087
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype    
---  ------            --------------  -----    
 0   date              480 non-null    period[M]
 1   country           480 non-null    object   
 2   category          480 non-null    object   
 3   product           480 non-null    object   
 4   beef price/100kg  480 non-null    float64  
dtypes: float64(1), object(3), period[M](1)
memory usage: 22.5+ KB


In [13]:
beef_price["date"] = beef_price["date"].astype(str)
beef_price.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 480 entries, 24 to 2087
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   date              480 non-null    object 
 1   country           480 non-null    object 
 2   category          480 non-null    object 
 3   product           480 non-null    object 
 4   beef price/100kg  480 non-null    float64
dtypes: float64(1), object(4)
memory usage: 22.5+ KB


In [14]:
beef_price.shape

(480, 5)

In [15]:
beef_price["category"].unique()

array(['Bulls', 'Cows', 'Heifers', 'Steers', 'Young bulls'], dtype=object)

In [16]:
beef_price["country"].unique()

array(['Ireland', 'Spain'], dtype=object)

In [17]:
beef_price["product"].unique()

array(['B R3', 'D R3', 'E R3', 'C R3', 'A R3'], dtype=object)

#### 1.2 Beef production

In this section, the beef production dataset was processed for merging later. 

In [18]:
beef_production = pd.read_excel("beef_production_2017_2021.xlsx")
beef_production.head()

,Member State,Member State Code,Category,Year,Month,1000 Heads,kg/head
0,Ireland,IE,Bull,2021,January,15.67,376.515635
1,Ireland,IE,Bull,2021,February,14.85,383.164983
2,Ireland,IE,Bull,2021,March,14.19,394.644116
3,Ireland,IE,Bull,2021,April,11.95,403.347280
4,Ireland,IE,Bull,2021,May,15.61,393.978219


In [19]:
beef_production.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Member State       600 non-null    object 
 1   Member State Code  600 non-null    object 
 2   Category           600 non-null    object 
 3   Year               600 non-null    int64  
 4   Month              600 non-null    object 
 5   1000 Heads         600 non-null    float64
 6   kg/head            600 non-null    float64
dtypes: float64(2), int64(1), object(4)
memory usage: 32.9+ KB


In [20]:
beef_production.shape

(600, 7)

In [21]:
beef_production.isnull().sum()

Member State         0
Member State Code    0
Category             0
Year                 0
Month                0
1000 Heads           0
kg/head              0
dtype: int64

In [22]:
# create a new date column based on Year and Month column
beef_production["date"] = pd.to_datetime(beef_production["Year"].astype(str)  + beef_production["Month"], format = "%Y%B")
beef_production.head()

,Member State,Member State Code,Category,Year,Month,1000 Heads,kg/head,date
0,Ireland,IE,Bull,2021,January,15.67,376.515635,2021-01-01
1,Ireland,IE,Bull,2021,February,14.85,383.164983,2021-02-01
2,Ireland,IE,Bull,2021,March,14.19,394.644116,2021-03-01
3,Ireland,IE,Bull,2021,April,11.95,403.347280,2021-04-01
4,Ireland,IE,Bull,2021,May,15.61,393.978219,2021-05-01


In [23]:
# change the date format into year-month
beef_production["date"] = beef_production["date"].dt.date.apply(lambda x: x.strftime('%Y-%m'))
beef_production

,Member State,Member State Code,Category,Year,Month,1000 Heads,kg/head,date
0,Ireland,IE,Bull,2021,January,15.67,376.515635,2021-01
1,Ireland,IE,Bull,2021,February,14.85,383.164983,2021-02
2,Ireland,IE,Bull,2021,March,14.19,394.644116,2021-03
3,Ireland,IE,Bull,2021,April,11.95,403.347280,2021-04
4,Ireland,IE,Bull,2021,May,15.61,393.978219,2021-05
...,...,...,...,...,...,...,...,...
595,Spain,ES,Young cattle,2017,August,85.21,241.286234,2017-08
596,Spain,ES,Young cattle,2017,September,83.32,241.358617,2017-09
597,Spain,ES,Young cattle,2017,October,87.65,240.844267,2017-10
598,Spain,ES,Young cattle,2017,November,84.57,240.392574,2017-11


In [24]:
beef_production = beef_production.drop(columns = {"Member State Code", "Year", "Month"})
beef_production.head()

,Member State,Category,1000 Heads,kg/head,date
0,Ireland,Bull,15.67,376.515635,2021-01
1,Ireland,Bull,14.85,383.164983,2021-02
2,Ireland,Bull,14.19,394.644116,2021-03
3,Ireland,Bull,11.95,403.347280,2021-04
4,Ireland,Bull,15.61,393.978219,2021-05


In [25]:
print(beef_production["Member State"].unique())
print(beef_price["country"].unique())

['Ireland' 'Spain']
['Ireland' 'Spain']


In [26]:
print(beef_production["Category"].unique())
print(beef_price["category"].unique())

['Bull' 'Bullock' 'Cow' 'Heifer' 'Young cattle']
['Bulls' 'Cows' 'Heifers' 'Steers' 'Young bulls']


In [27]:
# align the unique values of category in beef production with beef price
beef_production["Category"].replace(["Bull", "Bullock", "Cow", "Heifer", "Young cattle"], 
                                    ["Bulls", "Steers", "Cows", "Heifers", "Young bulls"], inplace=True)
beef_production.head()

,Member State,Category,1000 Heads,kg/head,date
0,Ireland,Bulls,15.67,376.515635,2021-01
1,Ireland,Bulls,14.85,383.164983,2021-02
2,Ireland,Bulls,14.19,394.644116,2021-03
3,Ireland,Bulls,11.95,403.347280,2021-04
4,Ireland,Bulls,15.61,393.978219,2021-05


In [28]:
beef_production.columns = ["country", "category", "1000 heads", "kg/head", "date"]
new_order2 = [-1, 0, 1, 2, -2]
beef_production = beef_production[beef_production.columns[new_order2]]
beef_production.head()

,date,country,category,1000 heads,kg/head
0,2021-01,Ireland,Bulls,15.67,376.515635
1,2021-02,Ireland,Bulls,14.85,383.164983
2,2021-03,Ireland,Bulls,14.19,394.644116
3,2021-04,Ireland,Bulls,11.95,403.347280
4,2021-05,Ireland,Bulls,15.61,393.978219


In [29]:
beef_production.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        600 non-null    object 
 1   country     600 non-null    object 
 2   category    600 non-null    object 
 3   1000 heads  600 non-null    float64
 4   kg/head     600 non-null    float64
dtypes: float64(2), object(3)
memory usage: 23.6+ KB


In [30]:
beef_production.shape

(600, 5)

In [31]:
beef_production["country"].unique()

array(['Ireland', 'Spain'], dtype=object)

In [32]:
beef_production["category"].unique()

array(['Bulls', 'Steers', 'Cows', 'Heifers', 'Young bulls'], dtype=object)

#### 1.3 Beef feeding price

In this section, the beef feeding price of Ireland was processed for merging later.

In [33]:
beef_feeding_price = pd.read_csv("beef_feeding_price_Ireland_2017_2021.csv")
beef_feeding_price.head()

,STATISTIC Label,Month,Type of Feedstuff,UNIT,VALUE
0,Feed Stuff Price,2017 January,Cattle fattening nuts and cubes (13-15% protein),Euro per Tonne,262.44
1,Feed Stuff Price,2017 January,Cattle fattening meal (13-15% protein),Euro per Tonne,250.44
2,Feed Stuff Price,2017 February,Cattle fattening nuts and cubes (13-15% protein),Euro per Tonne,264.56
3,Feed Stuff Price,2017 February,Cattle fattening meal (13-15% protein),Euro per Tonne,248.93
4,Feed Stuff Price,2017 March,Cattle fattening nuts and cubes (13-15% protein),Euro per Tonne,265.00


In [34]:
beef_feeding_price.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140 entries, 0 to 139
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   STATISTIC Label    140 non-null    object 
 1   Month              140 non-null    object 
 2   Type of Feedstuff  140 non-null    object 
 3   UNIT               140 non-null    object 
 4   VALUE              140 non-null    float64
dtypes: float64(1), object(4)
memory usage: 5.6+ KB


In [35]:
from datetime import datetime

# create a function to convert dates
def date_conversion(el):
    return datetime.strptime(el, "%Y %B").strftime("%Y-%m")


# apply conversion to desired column and store output in a new column
beef_feeding_price["date"] = beef_feeding_price["Month"].apply(date_conversion)
beef_feeding_price.head()

,STATISTIC Label,Month,Type of Feedstuff,UNIT,VALUE,date
0,Feed Stuff Price,2017 January,Cattle fattening nuts and cubes (13-15% protein),Euro per Tonne,262.44,2017-01
1,Feed Stuff Price,2017 January,Cattle fattening meal (13-15% protein),Euro per Tonne,250.44,2017-01
2,Feed Stuff Price,2017 February,Cattle fattening nuts and cubes (13-15% protein),Euro per Tonne,264.56,2017-02
3,Feed Stuff Price,2017 February,Cattle fattening meal (13-15% protein),Euro per Tonne,248.93,2017-02
4,Feed Stuff Price,2017 March,Cattle fattening nuts and cubes (13-15% protein),Euro per Tonne,265.00,2017-03


In [36]:
beef_feeding_price = beef_feeding_price.drop(columns = {"STATISTIC Label", "Month", "UNIT"})
beef_feeding_price.head()

,Type of Feedstuff,VALUE,date
0,Cattle fattening nuts and cubes (13-15% protein),262.44,2017-01
1,Cattle fattening meal (13-15% protein),250.44,2017-01
2,Cattle fattening nuts and cubes (13-15% protein),264.56,2017-02
3,Cattle fattening meal (13-15% protein),248.93,2017-02
4,Cattle fattening nuts and cubes (13-15% protein),265.00,2017-03


In [37]:
beef_feeding_price.columns = ["feedstuff", "beef feeding price/tonne", "date"]
new_order3 = [2, 0, 1]
beef_feeding_price = beef_feeding_price[beef_feeding_price.columns[new_order3]]
beef_feeding_price.head()

,date,feedstuff,beef feeding price/tonne
0,2017-01,Cattle fattening nuts and cubes (13-15% protein),262.44
1,2017-01,Cattle fattening meal (13-15% protein),250.44
2,2017-02,Cattle fattening nuts and cubes (13-15% protein),264.56
3,2017-02,Cattle fattening meal (13-15% protein),248.93
4,2017-03,Cattle fattening nuts and cubes (13-15% protein),265.00


- Filter into two datasets for merging later.

In [38]:
feedstuff1 = beef_feeding_price.query("feedstuff == 'Cattle fattening nuts and cubes (13-15% protein)'")
feedstuff1.head()

,date,feedstuff,beef feeding price/tonne
0,2017-01,Cattle fattening nuts and cubes (13-15% protein),262.44
2,2017-02,Cattle fattening nuts and cubes (13-15% protein),264.56
4,2017-03,Cattle fattening nuts and cubes (13-15% protein),265.00
6,2017-04,Cattle fattening nuts and cubes (13-15% protein),265.39
8,2017-05,Cattle fattening nuts and cubes (13-15% protein),265.67


In [39]:
feedstuff2 = beef_feeding_price.query("feedstuff == 'Cattle fattening meal (13-15% protein)'")
feedstuff2.head()

,date,feedstuff,beef feeding price/tonne
1,2017-01,Cattle fattening meal (13-15% protein),250.44
3,2017-02,Cattle fattening meal (13-15% protein),248.93
5,2017-03,Cattle fattening meal (13-15% protein),249.00
7,2017-04,Cattle fattening meal (13-15% protein),249.44
9,2017-05,Cattle fattening meal (13-15% protein),249.75


#### 1.4 Pigmeat price

In this section, the pigmeat price was collected through API and then processed for merging later.

In [40]:
# Get a request from the europa api that filter out the pigmeat price of Spain and Ireland from 2017 to 2021
# Only extract the E pigclass using the unit of 100KG
api_pigprice = "https://ec.europa.eu/agrifood/api/pigmeat/prices"

pigprice = {"memberStateCodes" : "ES,IE",
            "pigClasses" : "E",
            "beginDate" : "02/01/2017",
            "endDate" : "02/01/2022",
            "unit" : "100 KG"
    } 

response_pigprice = requests.get(api_pigprice, params=pigprice)

print(response_pigprice.status_code)
print(response_pigprice.url)

200
https://ec.europa.eu/agrifood/api/pigmeat/prices?memberStateCodes=ES%2CIE&pigClasses=E&beginDate=02%2F01%2F2017&endDate=02%2F01%2F2022&unit=100+KG


In [41]:
# check the json file
response_pigprice.json()

[{'memberStateCode': 'IE',
  'memberStateName': 'Ireland',
  'beginDate': '27/12/2021',
  'endDate': '02/01/2022',
  'weekNumber': 52,
  'price': '€143,39',
  'unit': '100 KG',
  'pigClass': 'E'},
 {'memberStateCode': 'ES',
  'memberStateName': 'Spain',
  'beginDate': '27/12/2021',
  'endDate': '02/01/2022',
  'weekNumber': 52,
  'price': '€126,29',
  'unit': '100 KG',
  'pigClass': 'E'},
 {'memberStateCode': 'IE',
  'memberStateName': 'Ireland',
  'beginDate': '20/12/2021',
  'endDate': '26/12/2021',
  'weekNumber': 51,
  'price': '€143,57',
  'unit': '100 KG',
  'pigClass': 'E'},
 {'memberStateCode': 'ES',
  'memberStateName': 'Spain',
  'beginDate': '20/12/2021',
  'endDate': '26/12/2021',
  'weekNumber': 51,
  'price': '€127,28',
  'unit': '100 KG',
  'pigClass': 'E'},
 {'memberStateCode': 'IE',
  'memberStateName': 'Ireland',
  'beginDate': '13/12/2021',
  'endDate': '19/12/2021',
  'weekNumber': 50,
  'price': '€143,77',
  'unit': '100 KG',
  'pigClass': 'E'},
 {'memberStateCode'

In [42]:
# save the json text file into a new variable
pig_price = json.loads(response_pigprice.text)
print(type(pig_price))
print(pig_price)

<class 'list'>
[{'memberStateCode': 'IE', 'memberStateName': 'Ireland', 'beginDate': '27/12/2021', 'endDate': '02/01/2022', 'weekNumber': 52, 'price': '€143,39', 'unit': '100 KG', 'pigClass': 'E'}, {'memberStateCode': 'ES', 'memberStateName': 'Spain', 'beginDate': '27/12/2021', 'endDate': '02/01/2022', 'weekNumber': 52, 'price': '€126,29', 'unit': '100 KG', 'pigClass': 'E'}, {'memberStateCode': 'IE', 'memberStateName': 'Ireland', 'beginDate': '20/12/2021', 'endDate': '26/12/2021', 'weekNumber': 51, 'price': '€143,57', 'unit': '100 KG', 'pigClass': 'E'}, {'memberStateCode': 'ES', 'memberStateName': 'Spain', 'beginDate': '20/12/2021', 'endDate': '26/12/2021', 'weekNumber': 51, 'price': '€127,28', 'unit': '100 KG', 'pigClass': 'E'}, {'memberStateCode': 'IE', 'memberStateName': 'Ireland', 'beginDate': '13/12/2021', 'endDate': '19/12/2021', 'weekNumber': 50, 'price': '€143,77', 'unit': '100 KG', 'pigClass': 'E'}, {'memberStateCode': 'ES', 'memberStateName': 'Spain', 'beginDate': '13/12/2021

In [43]:
# write a pandas dataframe using the json text file list
df_pigprice = pd.DataFrame(pig_price)
df_pigprice.head()

,memberStateCode,memberStateName,beginDate,endDate,weekNumber,price,unit,pigClass
0,IE,Ireland,27/12/2021,02/01/2022,52,"€143,39",100 KG,E
1,ES,Spain,27/12/2021,02/01/2022,52,"€126,29",100 KG,E
2,IE,Ireland,20/12/2021,26/12/2021,51,"€143,57",100 KG,E
3,ES,Spain,20/12/2021,26/12/2021,51,"€127,28",100 KG,E
4,IE,Ireland,13/12/2021,19/12/2021,50,"€143,77",100 KG,E


In [44]:
df_pigprice.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 520 entries, 0 to 519
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   memberStateCode  520 non-null    object
 1   memberStateName  520 non-null    object
 2   beginDate        520 non-null    object
 3   endDate          520 non-null    object
 4   weekNumber       520 non-null    int64 
 5   price            520 non-null    object
 6   unit             520 non-null    object
 7   pigClass         520 non-null    object
dtypes: int64(1), object(7)
memory usage: 32.6+ KB


In [45]:
df_pigprice.shape

(520, 8)

In [46]:
df_pigprice.isnull().sum()

memberStateCode    0
memberStateName    0
beginDate          0
endDate            0
weekNumber         0
price              0
unit               0
pigClass           0
dtype: int64

In [47]:
# create another function for convert dates
def date_conversion2(el):
    return datetime.strptime(el, "%d/%m/%Y").strftime("%Y-%m")

df_pigprice["date"] = df_pigprice["beginDate"].apply(date_conversion2)
df_pigprice.head()

,memberStateCode,memberStateName,beginDate,endDate,weekNumber,price,unit,pigClass,date
0,IE,Ireland,27/12/2021,02/01/2022,52,"€143,39",100 KG,E,2021-12
1,ES,Spain,27/12/2021,02/01/2022,52,"€126,29",100 KG,E,2021-12
2,IE,Ireland,20/12/2021,26/12/2021,51,"€143,57",100 KG,E,2021-12
3,ES,Spain,20/12/2021,26/12/2021,51,"€127,28",100 KG,E,2021-12
4,IE,Ireland,13/12/2021,19/12/2021,50,"€143,77",100 KG,E,2021-12


In [48]:
# create a function to remove euro sign
def remove_euro(x):
    return float(x.replace("€", "").replace(",", "."))

# apply the function to remove
df_pigprice["price"] = df_pigprice["price"].apply(remove_euro)
df_pigprice

,memberStateCode,memberStateName,beginDate,endDate,weekNumber,price,unit,pigClass,date
0,IE,Ireland,27/12/2021,02/01/2022,52,143.39,100 KG,E,2021-12
1,ES,Spain,27/12/2021,02/01/2022,52,126.29,100 KG,E,2021-12
2,IE,Ireland,20/12/2021,26/12/2021,51,143.57,100 KG,E,2021-12
3,ES,Spain,20/12/2021,26/12/2021,51,127.28,100 KG,E,2021-12
4,IE,Ireland,13/12/2021,19/12/2021,50,143.77,100 KG,E,2021-12
...,...,...,...,...,...,...,...,...,...
515,ES,Spain,16/01/2017,22/01/2017,3,140.03,100 KG,E,2017-01
516,IE,Ireland,09/01/2017,15/01/2017,2,153.10,100 KG,E,2017-01
517,ES,Spain,09/01/2017,15/01/2017,2,139.70,100 KG,E,2017-01
518,IE,Ireland,02/01/2017,08/01/2017,1,153.27,100 KG,E,2017-01


In [49]:
# similarly to beef price, convert weekly price to average monthly price for merging later
df_pigprice_group = df_pigprice.groupby(["memberStateName", "date"]).transform("mean")
df_pigprice_group

,weekNumber,price
0,50.5,143.550
1,50.5,125.880
2,50.5,143.550
3,50.5,125.880
4,50.5,143.550
...,...,...
515,3.0,140.368
516,3.0,154.016
517,3.0,140.368
518,3.0,154.016


In [50]:
df_pigprice["pig price"] = df_pigprice_group["price"]
df_pigprice.head(10)

,memberStateCode,memberStateName,beginDate,endDate,weekNumber,price,unit,pigClass,date,pig price
0,IE,Ireland,27/12/2021,02/01/2022,52,143.39,100 KG,E,2021-12,143.550
1,ES,Spain,27/12/2021,02/01/2022,52,126.29,100 KG,E,2021-12,125.880
2,IE,Ireland,20/12/2021,26/12/2021,51,143.57,100 KG,E,2021-12,143.550
3,ES,Spain,20/12/2021,26/12/2021,51,127.28,100 KG,E,2021-12,125.880
4,IE,Ireland,13/12/2021,19/12/2021,50,143.77,100 KG,E,2021-12,143.550
5,ES,Spain,13/12/2021,19/12/2021,50,126.07,100 KG,E,2021-12,125.880
6,IE,Ireland,06/12/2021,12/12/2021,49,143.47,100 KG,E,2021-12,143.550
7,ES,Spain,06/12/2021,12/12/2021,49,123.88,100 KG,E,2021-12,125.880
8,IE,Ireland,29/11/2021,05/12/2021,48,143.65,100 KG,E,2021-11,143.442
9,ES,Spain,29/11/2021,05/12/2021,48,126.01,100 KG,E,2021-11,126.170


In [51]:
df_pigprice = df_pigprice.drop_duplicates(["memberStateName", "pig price"], keep = "last")
df_pigprice.head(20)

,memberStateCode,memberStateName,beginDate,endDate,weekNumber,price,unit,pigClass,date,pig price
6,IE,Ireland,06/12/2021,12/12/2021,49,143.47,100 KG,E,2021-12,143.5500
7,ES,Spain,06/12/2021,12/12/2021,49,123.88,100 KG,E,2021-12,125.8800
16,IE,Ireland,01/11/2021,07/11/2021,44,143.48,100 KG,E,2021-11,143.4420
17,ES,Spain,01/11/2021,07/11/2021,44,126.49,100 KG,E,2021-11,126.1700
24,IE,Ireland,04/10/2021,10/10/2021,40,149.77,100 KG,E,2021-10,147.7700
25,ES,Spain,04/10/2021,10/10/2021,40,138.84,100 KG,E,2021-10,133.1775
32,IE,Ireland,06/09/2021,12/09/2021,36,151.40,100 KG,E,2021-09,151.5025
33,ES,Spain,06/09/2021,12/09/2021,36,150.16,100 KG,E,2021-09,146.1300
42,IE,Ireland,02/08/2021,08/08/2021,31,159.36,100 KG,E,2021-08,155.7540
43,ES,Spain,02/08/2021,08/08/2021,31,159.66,100 KG,E,2021-08,155.6220


In [52]:
df_pigprice = df_pigprice.drop(columns = {"memberStateCode", "beginDate", "endDate",
                                                       "weekNumber", "price", "unit", "pigClass"})
df_pigprice.head()

,memberStateName,date,pig price
6,Ireland,2021-12,143.550
7,Spain,2021-12,125.880
16,Ireland,2021-11,143.442
17,Spain,2021-11,126.170
24,Ireland,2021-10,147.770


In [53]:
df_pigprice.columns = ["country", "date", "pig price/100kg"]
new_order4 = [1, 0, 2]
df_pigprice = df_pigprice[df_pigprice.columns[new_order4]]
df_pigprice.head()

,date,country,pig price/100kg
6,2021-12,Ireland,143.550
7,2021-12,Spain,125.880
16,2021-11,Ireland,143.442
17,2021-11,Spain,126.170
24,2021-10,Ireland,147.770


In [54]:
df_pigprice.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 120 entries, 6 to 519
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   date             120 non-null    object 
 1   country          120 non-null    object 
 2   pig price/100kg  120 non-null    float64
dtypes: float64(1), object(2)
memory usage: 3.8+ KB


In [55]:
df_pigprice.shape

(120, 3)

In [56]:
df_pigprice["country"].unique()

array(['Ireland', 'Spain'], dtype=object)

#### 1.3 Data merging

In order to do an overall analysis, the four dataset were merged accordingly into two datasets for analysis:
- **ireland_beef** was merged to only analyse the beef price in Ireland and comparing to other factors;
- **ie_es_beef** was merged to compare the beef price between Ireland and Spain etc., however, the feedstuff price was excluded since the feedstuff price in Spain could not be collected.

#### 1.3.1 Merge to ireland_beef

In [57]:
ireland_beef_price = beef_price.query("country == 'Ireland'")
ireland_beef_price.head()

,date,country,category,product,beef price/100kg
24,2021-12,Ireland,Bulls,B R3,321.9675
26,2021-12,Ireland,Cows,D R3,378.2125
28,2021-12,Ireland,Heifers,E R3,431.5850
29,2021-12,Ireland,Steers,C R3,427.8350
30,2021-12,Ireland,Young bulls,A R3,407.8750


In [58]:
ireland_beef_production = beef_production.query("country == 'Ireland'")
ireland_beef_production.head()

,date,country,category,1000 heads,kg/head
0,2021-01,Ireland,Bulls,15.67,376.515635
1,2021-02,Ireland,Bulls,14.85,383.164983
2,2021-03,Ireland,Bulls,14.19,394.644116
3,2021-04,Ireland,Bulls,11.95,403.347280
4,2021-05,Ireland,Bulls,15.61,393.978219


In [59]:
ireland_pigmeat_price = df_pigprice.query("country == 'Ireland'")
ireland_pigmeat_price.head()

,date,country,pig price/100kg
6,2021-12,Ireland,143.5500
16,2021-11,Ireland,143.4420
24,2021-10,Ireland,147.7700
32,2021-09,Ireland,151.5025
42,2021-08,Ireland,155.7540


In [60]:
from functools import reduce

# define the datasets that need to be merged
df_merge = [ireland_beef_price, ireland_beef_production]

# merge datasets
ireland_beef = reduce(lambda left, right: pd.merge(left, right, on = ["date", "category"]), df_merge)
ireland_beef

,date,country_x,category,product,beef price/100kg,country_y,1000 heads,kg/head
0,2021-12,Ireland,Bulls,B R3,321.9675,Ireland,15.52,371.134021
1,2021-12,Ireland,Cows,D R3,378.2125,Ireland,33.75,297.777778
2,2021-12,Ireland,Heifers,E R3,431.5850,Ireland,45.77,305.003277
3,2021-12,Ireland,Steers,C R3,427.8350,Ireland,45.87,343.579682
4,2021-12,Ireland,Young bulls,A R3,407.8750,Ireland,0.19,210.526316
...,...,...,...,...,...,...,...,...
295,2017-01,Ireland,Bulls,B R3,300.2640,Ireland,24.61,371.800081
296,2017-01,Ireland,Cows,D R3,321.6440,Ireland,27.80,317.266187
297,2017-01,Ireland,Heifers,E R3,383.6880,Ireland,50.57,311.647222
298,2017-01,Ireland,Steers,C R3,370.5560,Ireland,46.02,351.803564


In [61]:
df_merge2 = [ireland_beef, ireland_pigmeat_price, feedstuff1, feedstuff2]

ireland_beef = reduce(lambda left, right: pd.merge(left, right, on = ["date"]), df_merge2)
ireland_beef

,date,country_x,category,product,beef price/100kg,country_y,1000 heads,kg/head,country,pig price/100kg,feedstuff_x,beef feeding price/tonne_x,feedstuff_y,beef feeding price/tonne_y
0,2021-12,Ireland,Bulls,B R3,321.9675,Ireland,15.52,371.134021,Ireland,143.550,Cattle fattening nuts and cubes (13-15% protein),345.13,Cattle fattening meal (13-15% protein),339.30
1,2021-12,Ireland,Cows,D R3,378.2125,Ireland,33.75,297.777778,Ireland,143.550,Cattle fattening nuts and cubes (13-15% protein),345.13,Cattle fattening meal (13-15% protein),339.30
2,2021-12,Ireland,Heifers,E R3,431.5850,Ireland,45.77,305.003277,Ireland,143.550,Cattle fattening nuts and cubes (13-15% protein),345.13,Cattle fattening meal (13-15% protein),339.30
3,2021-12,Ireland,Steers,C R3,427.8350,Ireland,45.87,343.579682,Ireland,143.550,Cattle fattening nuts and cubes (13-15% protein),345.13,Cattle fattening meal (13-15% protein),339.30
4,2021-12,Ireland,Young bulls,A R3,407.8750,Ireland,0.19,210.526316,Ireland,143.550,Cattle fattening nuts and cubes (13-15% protein),345.13,Cattle fattening meal (13-15% protein),339.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,2017-01,Ireland,Bulls,B R3,300.2640,Ireland,24.61,371.800081,Ireland,154.016,Cattle fattening nuts and cubes (13-15% protein),262.44,Cattle fattening meal (13-15% protein),250.44
296,2017-01,Ireland,Cows,D R3,321.6440,Ireland,27.80,317.266187,Ireland,154.016,Cattle fattening nuts and cubes (13-15% protein),262.44,Cattle fattening meal (13-15% protein),250.44
297,2017-01,Ireland,Heifers,E R3,383.6880,Ireland,50.57,311.647222,Ireland,154.016,Cattle fattening nuts and cubes (13-15% protein),262.44,Cattle fattening meal (13-15% protein),250.44
298,2017-01,Ireland,Steers,C R3,370.5560,Ireland,46.02,351.803564,Ireland,154.016,Cattle fattening nuts and cubes (13-15% protein),262.44,Cattle fattening meal (13-15% protein),250.44


In [62]:
# combine category and product columns into one column
ireland_beef["category"] = ireland_beef["category"] + " (" + ireland_beef["product"] + ")"
ireland_beef

,date,country_x,category,product,beef price/100kg,country_y,1000 heads,kg/head,country,pig price/100kg,feedstuff_x,beef feeding price/tonne_x,feedstuff_y,beef feeding price/tonne_y
0,2021-12,Ireland,Bulls (B R3),B R3,321.9675,Ireland,15.52,371.134021,Ireland,143.550,Cattle fattening nuts and cubes (13-15% protein),345.13,Cattle fattening meal (13-15% protein),339.30
1,2021-12,Ireland,Cows (D R3),D R3,378.2125,Ireland,33.75,297.777778,Ireland,143.550,Cattle fattening nuts and cubes (13-15% protein),345.13,Cattle fattening meal (13-15% protein),339.30
2,2021-12,Ireland,Heifers (E R3),E R3,431.5850,Ireland,45.77,305.003277,Ireland,143.550,Cattle fattening nuts and cubes (13-15% protein),345.13,Cattle fattening meal (13-15% protein),339.30
3,2021-12,Ireland,Steers (C R3),C R3,427.8350,Ireland,45.87,343.579682,Ireland,143.550,Cattle fattening nuts and cubes (13-15% protein),345.13,Cattle fattening meal (13-15% protein),339.30
4,2021-12,Ireland,Young bulls (A R3),A R3,407.8750,Ireland,0.19,210.526316,Ireland,143.550,Cattle fattening nuts and cubes (13-15% protein),345.13,Cattle fattening meal (13-15% protein),339.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,2017-01,Ireland,Bulls (B R3),B R3,300.2640,Ireland,24.61,371.800081,Ireland,154.016,Cattle fattening nuts and cubes (13-15% protein),262.44,Cattle fattening meal (13-15% protein),250.44
296,2017-01,Ireland,Cows (D R3),D R3,321.6440,Ireland,27.80,317.266187,Ireland,154.016,Cattle fattening nuts and cubes (13-15% protein),262.44,Cattle fattening meal (13-15% protein),250.44
297,2017-01,Ireland,Heifers (E R3),E R3,383.6880,Ireland,50.57,311.647222,Ireland,154.016,Cattle fattening nuts and cubes (13-15% protein),262.44,Cattle fattening meal (13-15% protein),250.44
298,2017-01,Ireland,Steers (C R3),C R3,370.5560,Ireland,46.02,351.803564,Ireland,154.016,Cattle fattening nuts and cubes (13-15% protein),262.44,Cattle fattening meal (13-15% protein),250.44


In [63]:
# drop the duplicated and unnecessary columns
ireland_beef = ireland_beef.drop(columns = {"product", "country_y", "country"})
ireland_beef.head()

,date,country_x,category,beef price/100kg,1000 heads,kg/head,pig price/100kg,feedstuff_x,beef feeding price/tonne_x,feedstuff_y,beef feeding price/tonne_y
0,2021-12,Ireland,Bulls (B R3),321.9675,15.52,371.134021,143.55,Cattle fattening nuts and cubes (13-15% protein),345.13,Cattle fattening meal (13-15% protein),339.3
1,2021-12,Ireland,Cows (D R3),378.2125,33.75,297.777778,143.55,Cattle fattening nuts and cubes (13-15% protein),345.13,Cattle fattening meal (13-15% protein),339.3
2,2021-12,Ireland,Heifers (E R3),431.5850,45.77,305.003277,143.55,Cattle fattening nuts and cubes (13-15% protein),345.13,Cattle fattening meal (13-15% protein),339.3
3,2021-12,Ireland,Steers (C R3),427.8350,45.87,343.579682,143.55,Cattle fattening nuts and cubes (13-15% protein),345.13,Cattle fattening meal (13-15% protein),339.3
4,2021-12,Ireland,Young bulls (A R3),407.8750,0.19,210.526316,143.55,Cattle fattening nuts and cubes (13-15% protein),345.13,Cattle fattening meal (13-15% protein),339.3


In [64]:
ireland_beef.columns = ["date", "country", "category", "beef price(€/100kg)", "1000 heads", "kg/head", 
                              "pigmeat price(€/100kg)", "feedstuff1", "cattle fattening nuts and cubes price(€/tonne)",
                              "feedstuff2", "cattle fattening meal(€/tonne)"]
ireland_beef.head()

,date,country,category,beef price(€/100kg),1000 heads,kg/head,pigmeat price(€/100kg),feedstuff1,cattle fattening nuts and cubes price(€/tonne),feedstuff2,cattle fattening meal(€/tonne)
0,2021-12,Ireland,Bulls (B R3),321.9675,15.52,371.134021,143.55,Cattle fattening nuts and cubes (13-15% protein),345.13,Cattle fattening meal (13-15% protein),339.3
1,2021-12,Ireland,Cows (D R3),378.2125,33.75,297.777778,143.55,Cattle fattening nuts and cubes (13-15% protein),345.13,Cattle fattening meal (13-15% protein),339.3
2,2021-12,Ireland,Heifers (E R3),431.5850,45.77,305.003277,143.55,Cattle fattening nuts and cubes (13-15% protein),345.13,Cattle fattening meal (13-15% protein),339.3
3,2021-12,Ireland,Steers (C R3),427.8350,45.87,343.579682,143.55,Cattle fattening nuts and cubes (13-15% protein),345.13,Cattle fattening meal (13-15% protein),339.3
4,2021-12,Ireland,Young bulls (A R3),407.8750,0.19,210.526316,143.55,Cattle fattening nuts and cubes (13-15% protein),345.13,Cattle fattening meal (13-15% protein),339.3


In [65]:
# drop the two distinctive feedstuff columns
ireland_beef = ireland_beef.drop(columns = {"feedstuff1", "feedstuff2"})
ireland_beef.head(10)

,date,country,category,beef price(€/100kg),1000 heads,kg/head,pigmeat price(€/100kg),cattle fattening nuts and cubes price(€/tonne),cattle fattening meal(€/tonne)
0,2021-12,Ireland,Bulls (B R3),321.9675,15.52,371.134021,143.550,345.13,339.3
1,2021-12,Ireland,Cows (D R3),378.2125,33.75,297.777778,143.550,345.13,339.3
2,2021-12,Ireland,Heifers (E R3),431.5850,45.77,305.003277,143.550,345.13,339.3
3,2021-12,Ireland,Steers (C R3),427.8350,45.87,343.579682,143.550,345.13,339.3
4,2021-12,Ireland,Young bulls (A R3),407.8750,0.19,210.526316,143.550,345.13,339.3
5,2021-11,Ireland,Bulls (B R3),312.5720,14.58,377.914952,143.442,333.38,330.1
6,2021-11,Ireland,Cows (D R3),366.9300,39.37,304.038608,143.442,333.38,330.1
7,2021-11,Ireland,Heifers (E R3),422.9340,55.18,304.458137,143.442,333.38,330.1
8,2021-11,Ireland,Steers (C R3),417.3040,61.99,352.153573,143.442,333.38,330.1
9,2021-11,Ireland,Young bulls (A R3),401.7100,0.17,235.294118,143.442,333.38,330.1


In [69]:
# Group the data by the 'category' column
beef_group = ireland_beef.groupby("category")

# Create a list to store the dataframes for each group
beef_list = []

# Iterate over the groups and create a separate dataframe for each group
for name, group in groups:
    beef_group = pd.DataFrame(group)
    beef_list.append(beef_group)

# Concatenate the dataframes into a single dataframe
ireland_beef_merged = reduce(lambda left, right: pd.merge(left, right, on = ["date"]), df_list)

ireland_beef_merged

,date,country_x,category_x,beef price(€/100kg)_x,1000 heads_x,kg/head_x,pigmeat price(€/100kg)_x,cattle fattening nuts and cubes price(€/tonne)_x,cattle fattening meal(€/tonne)_x,country_y,...,cattle fattening nuts and cubes price(€/tonne)_y,cattle fattening meal(€/tonne)_y,country,category,beef price(€/100kg),1000 heads,kg/head,pigmeat price(€/100kg),cattle fattening nuts and cubes price(€/tonne),cattle fattening meal(€/tonne)
0,2021-12,Ireland,Bulls (B R3),321.9675,15.52,371.134021,143.5500,345.13,339.30,Ireland,...,345.13,339.30,Ireland,Young bulls (A R3),407.8750,0.19,210.526316,143.5500,345.13,339.30
1,2021-11,Ireland,Bulls (B R3),312.5720,14.58,377.914952,143.4420,333.38,330.10,Ireland,...,333.38,330.10,Ireland,Young bulls (A R3),401.7100,0.17,235.294118,143.4420,333.38,330.10
2,2021-10,Ireland,Bulls (B R3),294.2975,9.12,391.447368,147.7700,329.13,325.73,Ireland,...,329.13,325.73,Ireland,Young bulls (A R3),400.1550,0.16,187.500000,147.7700,329.13,325.73
3,2021-09,Ireland,Bulls (B R3),297.3100,9.32,390.557940,151.5025,324.56,320.40,Ireland,...,324.56,320.40,Ireland,Young bulls (A R3),401.0025,0.06,166.666667,151.5025,324.56,320.40
4,2021-08,Ireland,Bulls (B R3),297.7820,9.23,396.533044,155.7540,317.75,310.73,Ireland,...,317.75,310.73,Ireland,Young bulls (A R3),402.5800,0.07,142.857143,155.7540,317.75,310.73
5,2021-07,Ireland,Bulls (B R3),303.5950,12.00,394.166667,160.7775,317.38,310.47,Ireland,...,317.38,310.47,Ireland,Young bulls (A R3),413.5900,0.10,200.000000,160.7775,317.38,310.47
6,2021-06,Ireland,Bulls (B R3),319.1475,15.52,384.664948,167.2300,313.44,308.13,Ireland,...,313.44,308.13,Ireland,Young bulls (A R3),407.1125,0.13,230.769231,167.2300,313.44,308.13
7,2021-05,Ireland,Bulls (B R3),325.5100,15.61,393.978219,165.7720,310.75,304.73,Ireland,...,310.75,304.73,Ireland,Young bulls (A R3),397.1100,0.41,121.951220,165.7720,310.75,304.73
8,2021-04,Ireland,Bulls (B R3),326.8400,11.95,403.347280,162.6050,305.38,299.20,Ireland,...,305.38,299.20,Ireland,Young bulls (A R3),383.5850,0.74,121.621622,162.6050,305.38,299.20
9,2021-03,Ireland,Bulls (B R3),323.6140,14.19,394.644116,155.2700,301.00,294.87,Ireland,...,301.00,294.87,Ireland,Young bulls (A R3),363.9740,3.68,100.543478,155.2700,301.00,294.87


In [70]:
ireland_beef_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60 entries, 0 to 59
Data columns (total 41 columns):
 #   Column                                            Non-Null Count  Dtype  
---  ------                                            --------------  -----  
 0   date                                              60 non-null     object 
 1   country_x                                         60 non-null     object 
 2   category_x                                        60 non-null     object 
 3   beef price(€/100kg)_x                             60 non-null     float64
 4   1000 heads_x                                      60 non-null     float64
 5   kg/head_x                                         60 non-null     float64
 6   pigmeat price(€/100kg)_x                          60 non-null     float64
 7   cattle fattening nuts and cubes price(€/tonne)_x  60 non-null     float64
 8   cattle fattening meal(€/tonne)_x                  60 non-null     float64
 9   country_y              

In [67]:
# Create a wide-form dataframe with separate columns for each category
df_wide = ireland_beef.pivot_table(index=["date", "country"], 
                                   columns="category", 
                                   values=["beef price(€/100kg)", "1000 heads", "kg/head"])

# Extract a dataframe for a particular category
df_category = df_wide.xs("Bulls (B R3)", level="category", axis=1)

df_wide
df_category.head()

,,1000 heads,beef price(€/100kg),kg/head
date,country,,,
2017-01,Ireland,24.61,300.2640,371.800081
2017-02,Ireland,22.40,317.1275,376.785714
2017-03,Ireland,17.83,328.5500,382.501402
2017-04,Ireland,15.32,338.1575,385.770235
2017-05,Ireland,21.47,333.0160,375.873312


In [68]:
df_merge3 = [beef_price_analysis, feedstuff1, feedstuff2]

beef_price_analysis = reduce(lambda left, right: pd.merge(left, right, on = ["date"]), df_merge3)
beef_price_analysis

NameError: name 'beef_price_analysis' is not defined

In [ ]:
beef_price_analysis = beef_price_analysis.sort_values(by = ["date"], ascending = True)
beef_price_analysis.head(15)

- **From here, the final dataset is almost ready. There are just few more modifications to do as follow.**

In [ ]:
# combine category and product columns into one column
beef_price_analysis["category"] = beef_price_analysis["category"] + " (" + beef_price_analysis["product"] + ")"
beef_price_analysis

In [ ]:
# drop the duplicated and unnecessary columns
beef_price_analysis = beef_price_analysis.drop(columns = {"product", "country_y", "country"})
beef_price_analysis.head()

In [ ]:
# rename the columns
beef_price_analysis.columns = ["date", "country", "category", "beef price(€/100kg)", "1000 heads", "kg/head", 
                              "pigmeat price(€/100kg)", "feedstuff1", "cattle fattening nuts and cubes price(€/tonne)",
                              "feedstuff2", "cattle fattening meal(€/tonne)"]
beef_price_analysis.head()

In [ ]:
# drop the two distinctive feedstuff columns
beef_price_analysis = beef_price_analysis.drop(columns = {"feedstuff1", "feedstuff2"})
beef_price_analysis.head()

In [ ]:
# lastly, replace the two feedstuff price in Spain with value 0 since we couldn't get data for it
# columns_to_update = ["cattle fattening nuts and cubes price(€/tonne)", "cattle fattening meal(€/tonne)"]

# beef_price_analysis.loc[beef_price_analysis["country"] == "Spain", columns_to_update] = 0
# beef_price_analysis.head()

In [ ]:
beef_price_analysis.shape

In [ ]:
beef_price_analysis.info()

In [ ]:
beef_price_analysis["date"] = pd.to_datetime(beef_price_analysis["date"])
beef_price_analysis.info()

### 2. EDA

- Looking into Ireland only.

In [ ]:
ireland = beef_price_analysis.query("country == 'Ireland'")
ireland.head()

In [ ]:
from dataprep.eda import plot
plot(ireland)

In [ ]:
plot(ireland, "beef price(€/100kg)", "pigmeat price(€/100kg)")

In [ ]:
from dataprep.eda import plot_correlation
plot_correlation(ireland)

In [ ]:
corrmat = ireland.corr()
f, ax = plt.subplots(figsize = (8, 8))
fig1 = sns.heatmap(corrmat, vmax = 1, cmap = "Paired", square = True, annot = True);

In [ ]:
# import bokeh
# from bokeh.plotting import figure, show
# # create a new plot with a title and axis labels
# p = figure(title = "Beef Price in Ireland from 2017 to 2021", x_axis_label = "Date", 
#            y_axis_label = "Beef price(€/100kg)", sizing_mode="stretch_width", height = 400)

# # add a line renderer with legend and line thickness
# p.line(x = ireland["date"], y = ireland["beef price(€/100kg)"], legend_label = "beef price", line_width = 2)

# # show the results
# show(p)

In [ ]:
import plotly.express as px

fig = px.line(ireland["category"], x = "date", y = "beef price(€/100kg)")
fig.show()

###  Sentimental analysis

In [72]:
from dotenv import dotenv_values

config = dotenv_values(".env")

## References


- <p> https://stackoverflow.com/questions/50663700/convert-year-and-month-name-into-datetime-column-for-pandas-dataframe <p>
- <p> https://stackoverflow.com/questions/50511640/how-to-convert-string-to-datetime-without-days-in-pandas <p>
- <p> https://stackoverflow.com/questions/56723193/convert-stringdec-2018-to-datetime <p>